In [15]:
# pip install pydicom
# pip install numpy
# pip install matplotlib
# pip install opencv-python
# pip install tqdm
# pip install pandas
# pip install glob2
# pip install pylibjpeg
# pip install pylibjpeg-libjpeg==1.3.1

In [16]:
import DicomPipeLine 
import cv2

In [17]:
main_data_source_path = r'/app/Data Pipeline/DataBase_1'

In [18]:
preprocessed_data_path = r'/app/Data Pipeline/TestDataBase_2'

In [19]:
sub_folders_to_scan_for = ['2','3','4','5']

In [20]:
path_obj = DicomPipeLine.Path_Settings(main_data_source_path, preprocessed_data_path, sub_folders_to_scan_for)

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 386.79it/s]

- Found 0 folders with a total of 0 scans


In [21]:
path_obj.source_folders

[]

In [22]:
path_obj.save_folders

[<DirEntry '1'>, <DirEntry '2'>, <DirEntry '3'>, <DirEntry '4'>]

In [28]:
# these are the attributes to extract from each patient
dcm_attributes_level_0 = [
    'PatientBirthDate',
    'PatientSex',
    'PatientSize',
    'PatientWeight',
    'StudyDate', 
    'StudyTime',
    'InstitutionAddress', 
    'InstitutionName',
    'Manufacturer', 
    'ManufacturerModelName'
]

# these are the attributes to extract from each scan per patient
dcm_attributes_level_1 = [
    'AcquisitionDate', 
    'AcquisitionTime', 
    'PatientPosition',
    'SeriesDescription',
    'SeriesDate',
    'SeriesTime'
]

# these are the attributes to extract from each frame per scan 
dcm_attributes_level_2 = [
    'Rows',
    'Columns',
    'InstanceNumber',
    'PixelSpacing',
    'ImagePositionPatient',
    'ImageOrientationPatient',
    'SliceLocation',
    'SliceThickness'
]

# specify all 3 levels of attributes in a array
dcm_attributes = [dcm_attributes_level_0, dcm_attributes_level_1, dcm_attributes_level_2]


In [29]:
dcm_to_numpy_obj = DicomPipeLine.DCM_Input_To_NPY_Output(path_obj, dcm_attributes)

In [30]:
path_obj.Refresh()

100%|██████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 10195.39it/s]

- Found 49 folders with a total of 191 scans


In [31]:
dcm_to_numpy_obj.iterate(2)

- processing folder F:\DICOM data\1.3.12.2.1107.5.1.4.11050.30000021082603485362500000022


100%|███████████████████████████████████████████████████████████████████████████████| 329/329 [00:11<00:00, 28.58it/s]


- Saving Processed scans ['PET WB  (AC)', 'CT PLAIN', 'CT ARTERIAL', 'CT VENOUS'] ....


KeyboardInterrupt: 

In [11]:
streamer = DicomPipeLine.Stream_Data(path_obj)

In [12]:
import time
import numpy as np
from skimage.transform import resize

def normalize(img_array):
    return cv2.normalize(img_array, dst=None, alpha=0, beta=65535, norm_type=cv2.NORM_MINMAX)

In [ ]:

cv2.namedWindow('fused',cv2.WINDOW_NORMAL)
cv2.resizeWindow('fused', 600,600)

for ct_frame, pt_frame in zip(streamer.iter('1', 'CT PLAIN'), streamer.iter('1', 'PET WB  (AC)')):
    
    ct_frame = normalize(ct_frame)
    ct_frame = cv2.resize(ct_frame, pt_frame.shape)
    pt_frame = normalize(pt_frame)
    
    cv2.imshow('CT Image',ct_frame)
    cv2.imshow('PET Image', pt_frame)
    
    fused = (normalize(ct_frame + pt_frame))
    
    fused = resize(fused, (600, 600), anti_aliasing=True)
    
    cv2.imshow('fused', fused)
    
    time.sleep(.1)
    cv2.waitKey(4)

cv2.destroyAllWindows()


In [ ]:
# prev_folder = None
# prev_scan = None
# for img_array, folder_name, scan_name in streamer.iterate_image_data(0, 1):
    
#     img_array = cv2.normalize(img_array, dst=None, alpha=0, beta=65535, norm_type=cv2.NORM_MINMAX)
#     cv2.imshow('View Window',img_array)
    
#     if folder_name != prev_folder or prev_scan != scan_name:
#         print(f'folder : {folder_name} | Scan : {scan_name}')
#         prev_folder = folder_name
#         prev_scan = scan_name

#     cv2.waitKey(2)